# Detecting Synthetic Faces

Library used for backbone of detection pipeline is Fast AI. Refer to [documentation](https://docs.fast.ai/) for more information.

### Importing Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%env TORCH_HOME=drive/My Drive/models

env: TORCH_HOME=drive/My Drive/models


In [ ]:
!cp '/content/drive/My Drive/new_data/test.zip' .
!unzip -q test.zip
!rm test.zip
!cp '/content/drive/My Drive/new_data/val_psis.zip' .
!unzip -q val_psis.zip
!rm val_psis.zip
!cp '/content/drive/My Drive/new_data/test_psis.zip' .
!unzip -q test_psis.zip
!rm test_psis.zip

In [ ]:
!cp '/content/drive/My Drive/new_data/128.zip' .
!unzip -q 128.zip
!rm 128.zip

In [ ]:
!cp '/content/drive/My Drive/new_data/64.zip' .
!unzip -q 64.zip
!rm 64.zip

### Training

In [ ]:
%%capture
!pip install pretrainedmodels

In [ ]:
from fastai.vision import *
from fastai.vision.models import cadene_models
from fastai.metrics import error_rate
from fastai.callbacks import *
from fastai.distributed import *
import torch
import torch.nn as nn

In [ ]:
paths=['64','128']

In [ ]:
idx_paths = "0"  #@param [0, 1, 2, 3]
path=paths[int(idx_paths)]
bs=100
print(path)

64


In [ ]:
size=int(path)
print(size)

64


In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
#For Training
data=ImageDataBunch.from_folder(path,train='train',valid='val',seed=42,bs=bs).normalize(imagenet_stats)

In [ ]:
data.show_batch()

In [ ]:
from fastai.vision import *
resnet_models=[models.resnet18,models.resnet34,models.resnet50,models.resnet101,models.resnet152]
vgg_models=[models.vgg11_bn,models.vgg13_bn,models.vgg16_bn,models.vgg19_bn]
densenet_models=[models.densenet121,models.densenet161,models.densenet169,models.densenet201]
models=[resnet_models,vgg_models,densenet_models]

In [ ]:
type_idx="1" #@param [0, 1, 2, 3]
model_idx="0" #@param [0, 1, 2, 3, 4, 5, 6, 7 ,8 ,9 ,10]
model=models[int(type_idx)][int(model_idx)]
model_name=str(model).split()[1]
print(model_name)

In [ ]:
learn = cnn_learner(data, model, metrics=[error_rate,accuracy],path='drive/My Drive/models/',pretrained=True)#.to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(10,max_lr=1e-3,callbacks=[SaveModelCallback(learn,monitor='valid_loss',mode='min',name=f"{model_name}_{path}_1"),ShowGraph(learn)])

In [ ]:
%%capture
learn.load(f'{model_name}_{path}_1')

In [ ]:
learn.validate()

In [ ]:
psis_val=['50','60','70','80','90']
psis_test=['55','65','75','85','95']
print("test")
for psi in psis_test:
  data_val=ImageDataBunch.from_folder(path="test_psis",train='test_'+psi,valid='test_'+psi,seed=42,bs=bs,size=(size,size)).normalize(imagenet_stats)
  learn.data=data_val
  print(psi,end=' ')
  x=learn.validate()
  print(x)
print("total test")
data_test=ImageDataBunch.from_folder(path="test",train='test',valid='test',seed=42,bs=bs,size=(size,size)).normalize(imagenet_stats)
learn.data=data_test
x=learn.validate()
print(x)



### Training Loop for all models

In [ ]:
from fastai.vision import *
resnet_models=[models.resnet18,models.resnet34,models.resnet50,models.resnet101,models.resnet152]
vgg_models=[models.vgg11_bn,models.vgg13_bn,models.vgg16_bn,models.vgg19_bn]
densenet_models=[models.densenet121,models.densenet161,models.densenet169,models.densenet201]
models=[resnet_models,vgg_models,densenet_models]

In [ ]:
#Training loop
from IPython.display import clear_output
#for model_category in models:
for model in densenet_models:
  model_name=str(model).split()[1]
  print(model_name)
  learn = cnn_learner(data, model, metrics=[error_rate,accuracy],path='drive/My Drive/models/',pretrained=True)#.to_fp16()
  learn.fit_one_cycle(20,max_lr=1.2e-3,callbacks=[SaveModelCallback(learn,monitor='valid_loss',mode='min',name=f"{model_name}_{path}_1"),ShowGraph(learn)])
  psis_test=['55','65','75','85','95']
  print("test")
  for psi in psis_test:
    data_val=ImageDataBunch.from_folder(path="test_psis",train='test_'+psi,valid='test_'+psi,seed=42,bs=bs,size=(size,size)).normalize(imagenet_stats)
    learn.data=data_val
    print(psi,end=' ')
    x=learn.validate()
    print(x)
  print("total test")
  data_test=ImageDataBunch.from_folder(path="test",train='test',valid='test',seed=42,bs=bs,size=(size,size)).normalize(imagenet_stats)
  learn.data=data_test
  x=learn.validate()
  print(x)
  #clear_output(wait=True)

In [ ]:
#Testing loop
for model in resnet_models:
  model_name=str(model).split()[1]
  print(model_name)
  learn = cnn_learner(data, model, metrics=[error_rate,accuracy],path='drive/My Drive/models/',pretrained=True)#.to_fp16()
  learn.load(f'{model_name}_{path}_1')
  psis_test=['55','65','75','85','95']
  print("test")
  for psi in psis_test:
    data_val=ImageDataBunch.from_folder(path="test_psis",train='test_'+psi,valid='test_'+psi,seed=42,bs=bs,size=(size,size)).normalize(imagenet_stats)
    learn.data=data_val
    print(psi,end=' ')
    x=learn.validate()
    print(x)
  print("total test")
  data_test=ImageDataBunch.from_folder(path="test",train='test',valid='test',seed=42,bs=bs,size=(size,size)).normalize(imagenet_stats)
  learn.data=data_test
  x=learn.validate()
  print(x)

resnet18
test
55 

[0.32156, tensor(0.1250), tensor(0.8750)]
65 

[0.3943863, tensor(0.1775), tensor(0.8225)]
75 

[0.55630124, tensor(0.2825), tensor(0.7175)]
85 

[0.66633314, tensor(0.3600), tensor(0.6400)]
95 

...

[0.34708512, tensor(0.1500), tensor(0.8500)]
75 

[0.48067582, tensor(0.2200), tensor(0.7800)]
85 

[0.65661865, tensor(0.3225), tensor(0.6775)]
95 

[0.7999374, tensor(0.4100), tensor(0.5900)]
total test


[0.5118756, tensor(0.2453), tensor(0.7548)]
